In [5]:
#%pip install --extra-index-url https://test.pypi.org/simple ../../EnergyPlus-OOEP/
#%pip install --extra-index-url https://test.pypi.org/simple git+https://github.com/BuildingGym/EnergyPlus-OOEP
#python3 -m pip install --extra-index-url https://test.pypi.org/simple git+https://github.com/BuildingGym/EnergyPlus-OOEP
#%pip install ipywidgets tqdm gymnasium torch ray[rllib]
#%pip install /home/user@AD/lab/EnergyPlus-Datasets/


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Processing /home/user@AD/lab/EnergyPlus-Datasets
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for energyplus-datasets: filename=energyplus_datasets-0.0.post1.dev4+g62b02ce-py3-none-any.whl size=41418456 sha256=507724d88d36d8d633637767cfa44e4266fc108ba76c5eb849cbe5c7ce4defc7
  Stored in directory: /home/user@AD/.cache/pip/wheels/88/39/1e/b1aa4c7ad5539f662fdf9962d941f2eba2871b1a3167609511
Successfully built energyplus-datasets
Note: you may need to restart the kernel to use updated packages.


Kickstart the simulation.

In [6]:
#import logging as _logging_
#_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

from energyplus.ooep.addons.progress import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        model=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-5' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyGym/.venv/lib/python3.11/site-packages/energyplus/ooep/utils/awaitables.py:68>>

method 1

In [7]:
import energyplus.ooep as _ooep_

obs = simulator.variables.getdefault(
    _ooep_.OutputVariable.Ref(
        type='People Air Temperature',
        key='CORE_MID',
    )
).value

simulator.variables.getdefault(
    _ooep_.Actuator.Ref(
        type='Zone Temperature Control',
        control_type='Heating Setpoint',
        key='CORE_MID',
    )
).value = 16
actuator = simulator.variables.getdefault(
    _ooep_.Actuator.Ref(
        type='Zone Temperature Control',
        control_type='Heating Setpoint',
        key='CORE_MID',
    )
).value 
print(obs,actuator)


24.713092250368426 16.0


method 2

In [8]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl import (
    VariableBox,
    SimulatorEnv,
)

from energyplus.ooep import (
    Actuator,
    OutputVariable,
)
from energyplus.ooep.addons.rl.gymnasium import ThinEnv

simulator.add(
    thinenv := ThinEnv(
        action_space=_gymnasium_.spaces.Dict({
                    'thermostat': VariableBox(
                        low=15., high=16.,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(Actuator.Ref(
                        type='Zone Temperature Control',
                        control_type='Heating Setpoint',
                        key='CORE_MID',
                    ))
                }),    
        observation_space=_gymnasium_.spaces.Dict({
            'temperature': VariableBox(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Ref(
                type='People Air Temperature',
                key='CORE_MID',
            )),
        }),
    )
)
obs = thinenv.observe()
act = thinenv.act({'thermostat': 16. })
print(obs,act)

ImportError: cannot import name 'VariableBox' from 'energyplus.ooep.addons.rl' (/home/user@AD/lab/EnergyGym/.venv/lib/python3.11/site-packages/energyplus/ooep/addons/rl/__init__.py)